In [ ]:
import pandas as pd
import numpy as np
from sglang import function, system, user, assistant, gen, set_default_backend, RuntimeEndpoint
import sglang as sgl
from torch import save


set_default_backend(RuntimeEndpoint("http://localhost:30000"))

@sgl.function
def text_qa(s, question):
    s += question
    s += sgl.gen("answer", stop=None, max_tokens=400,temperature=0)

# Step 1: Read the inputs.csv file and sample 50 entries
df_inputs = pd.read_csv('../2_train_set_creation/inputs.csv')
df_inputs = df_inputs

# Function to split DataFrame into chunks
def split_dataframe(df, chunk_size):
    # Use np.array_split to handle uneven sizes automatically
    chunks = np.array_split(df, len(df) // chunk_size + (1 if len(df) % chunk_size else 0))
    return chunks

# Split the DataFrame into chunks of size 5
df_chunks = split_dataframe(df_inputs, 2000)  # Adjust the chunk size as needed

for thing in df_chunks:
    print(len(thing))

In [ ]:
# Process each chunk
for i, df_chunk in enumerate(df_chunks):
    questions = df_chunk['input'].tolist()
    
    # Run batch inference for the current chunk
    states = text_qa.run_batch(
        [{"question": question} for question in questions],
        progress_bar=True
    )
    
    texts = []
    for state in states:
        for out in state.text_iter():
            texts.append(out)
    
    # Save the outputs of the current chunk to a separate file
    save(texts, f'../5_processing_extracted_data/outputs_chunk_{i+1}.pt')